<a href="https://colab.research.google.com/github/YellaCRE/DACON/blob/main/2022.04.19_DACON_%EC%86%8C%EB%93%9D%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycaret[full]

In [ ]:
!pip install -U pandas-profiling

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#데이터 불러오기

In [ ]:
import os
import pandas as pd
pd.set_option('mode.chained_assignment',  None)

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas_profiling

In [ ]:
#csv 형식으로 된 데이터 파일을 읽어옵니다. 
train = pd.read_csv('/content/drive/MyDrive/DACON/2022.04.19 소득예측/data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/DACON/2022.04.19 소득예측/data/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/DACON/2022.04.19 소득예측/data/sample_submission.csv')

#결측치 확인

In [ ]:
def check_missing_col(dataframe):
    missing_col = []
    counted_missing_col = 0
    for i, col in enumerate(dataframe.columns):
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            counted_missing_col += 1
            print(f'결측치가 있는 컬럼은: {col}입니다')
            print(f'해당 컬럼에 총 {missing_values}개의 결측치가 존재합니다.')
            missing_col.append([col, dataframe[col].dtype])
    if counted_missing_col == 0:
        print('결측치가 존재하지 않습니다')
    return missing_col

missing_col = check_missing_col(train)

결측치가 있는 컬럼은: workclass입니다
해당 컬럼에 총 1836개의 결측치가 존재합니다.
결측치가 있는 컬럼은: occupation입니다
해당 컬럼에 총 1843개의 결측치가 존재합니다.
결측치가 있는 컬럼은: native.country입니다
해당 컬럼에 총 583개의 결측치가 존재합니다.


In [ ]:
# 결측치를 처리하는 함수를 작성합니다.
def handle_na(data, missing_col):
    temp = data.copy()
    for col, dtype in missing_col:
        if dtype == 'O':
            # 범주형 feature가 결측치인 경우 해당 행들을 삭제해 주었습니다.
            temp = temp.dropna(subset=[col])
    return temp

train = handle_na(train, missing_col)

# 결측치 처리가 잘 되었는지 확인해 줍니다.
missing_col = check_missing_col(train) 

결측치가 존재하지 않습니다


#전처리

In [ ]:
df = pd.concat([train, test], axis=0)
df.head(5)

,id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,target
0,0,32,Private,309513,Assoc-acdm,12,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0.0
1,1,33,Private,205469,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,1.0
2,2,46,Private,149949,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0.0
3,3,23,Private,193090,Bachelors,13,Never-married,Adm-clerical,Own-child,White,Female,0,0,30,United-States,0.0
4,4,55,Private,60193,HS-grad,9,Divorced,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States,0.0


In [ ]:
df.profile_report()

In [ ]:
# Fnlwgt, education 제거
df = df.drop(['fnlwgt','education.num','relationship'], axis=1)

그룹화

In [ ]:
# sex
cond_0 = (df['sex'] == 'Male')
cond_1 = (df['sex'] != 'Male')
df.loc[(cond_0), 'sex'] = 0
df.loc[(cond_1), 'sex'] = 1

In [ ]:
# work class
govs = ['Federal-gov', 'Local-gov', 'State-gov']
private = ['Private', 'Self-emp-inc', 'Self-emp-not-inc']
no_pay = ['Never-worked', 'Without-pay']
for idx, row in df.iterrows():
    if row['workclass'] in govs:
        df.loc[idx, 'workclass'] = 0
    elif row['workclass'] in private:
        df.loc[idx, 'workclass'] = 1
    elif row['workclass'] in no_pay:
        df.loc[idx, 'workclass'] = 2
    else:
        df.loc[idx, 'workclass'] = np.nan

In [ ]:
# education
ct = round(pd.crosstab(train['education'], 
            train['target'],
            normalize='index',
            )*100,2)

ratio_dict = dict()
for _education in train['education'].unique():
    if ct.loc[_education, 1] >= 40:
        ratio_dict[_education] = 3
    elif (ct.loc[_education, 1] < 40) and (ct.loc[_education, 1] > 20):
        ratio_dict[_education] = 2
    elif (ct.loc[_education, 1] < 20) and (ct.loc[_education, 1] > 10):
        ratio_dict[_education] = 1        
    else:
        ratio_dict[_education] = 0

df['education'] = df['education'].map(ratio_dict)

In [ ]:
# martial.status
ct = round(pd.crosstab(train['marital.status'], 
            train['target'],
            normalize='index',
            )*100,2)

ratio_dict = dict()
for _marital in train['marital.status'].unique():
    if ct.loc[_marital, 1] >= 30:
        ratio_dict[_marital] = 1
    else:
        ratio_dict[_marital] = 0

df['marital.status'] = df['marital.status'].map(ratio_dict)

In [ ]:
# occupation
ct = round(pd.crosstab(train['occupation'], 
            train['target'],
            normalize='index',
            )*100,2)

ratio_dict = dict()
for _occupation in train['occupation'].unique()[:-1]: # nan은 별도로 추가하지 않습니다.
    if ct.loc[_occupation, 1] >= 40:
        ratio_dict[_occupation] = 2
    elif (ct.loc[_occupation, 1] < 40) and (ct.loc[_occupation, 1] > 15):
        ratio_dict[_occupation] = 1
    else:
        ratio_dict[_occupation] = 0

df['occupation'] = df['occupation'].map(ratio_dict)

In [ ]:
# hours.per.week
cond_0 = (df['hours.per.week'] < 40)
cond_1 = (df['hours.per.week'] >= 40)
df.loc[(cond_0), 'hours.per.week'] = 0
df.loc[(cond_1), 'hours.per.week'] = 1

In [ ]:
# native.country
ct = round(pd.crosstab(train['native.country'], 
            train['target'],
            normalize='index',
            )*100,2)

us_ratio = ct.loc['United-States', 1]
ratio_dict = dict()
for _country in train['native.country'].unique()[:-1]:
    if ct.loc[_country, 1] >= us_ratio:
        ratio_dict[_country] = 1
    else:
        ratio_dict[_country] = 0

df['native.country'] = df['native.country'].map(ratio_dict)

In [ ]:
train = df[df['target'].notnull()]
test = df[df['target'].isnull()].drop('target', axis=1)

scaling

In [ ]:
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import RobustScaler

con_feature = ['age','capital.gain','capital.loss']

train_feature, test_feature = train.loc[:, con_feature], test.loc[:, con_feature]
scaler = RobustScaler().fit(train_feature)
train.loc[:, con_feature] = scaler.transform(train_feature)
test.loc[:, con_feature] = scaler.transform(test_feature)

In [ ]:
df.profile_report()

#모델링

In [ ]:
from pycaret.classification import *
# 메세지가 뜨겠으나 사용에 문제는 없습니다.

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [ ]:
# 연속형 변수와 범주형 변수들을 리스트로 만들어둡니다.
cat_feature = list(set(list(train.columns)) - set(['id','target']) - set(con_feature))
print(con_feature, cat_feature, sep='\n')

['age', 'capital.gain', 'capital.loss']
['occupation', 'native.country', 'education', 'race', 'marital.status', 'workclass', 'hours.per.week', 'sex']


In [ ]:
# 이외에도 추가적인 파라미터들이 있습니다
s = setup(
    data = train,
    target = 'target',
    numeric_features = con_feature,
    categorical_features = cat_feature,
    train_size = 0.85,
    fold_strategy = 'stratifiedkfold',
    fold=5,
    fold_shuffle=True,
    data_split_stratify = True,
    fix_imbalance = True,
    ignore_low_variance=True,
    remove_outliers=True,
    remove_multicollinearity = True,
    multicollinearity_threshold = 0.9,
    feature_interaction=True,
    polynomial_features=True,
    combine_rare_levels = True,
    rare_level_threshold = 0.05,
    ignore_features = ['id'],
    silent = True,
    verbose = True,
    use_gpu = True
    )

# 지금은 사용하지 않을 지표인 kappa와 mcc를 평가지표에서 제거합니다.
remove_metric('kappa')
remove_metric('mcc')

,Description,Value
0,session_id,6243
1,Target,target
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(15081, 13)"
5,Missing Values,True
6,Numeric Features,3
7,Categorical Features,8
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
n_models = compare_models(n_select=5) # 성능이 가장 우수한 상위 5개 모델을 저장합니다.

,Model,Accuracy,AUC,Recall,Prec.,F1,TT (Sec)
catboost,CatBoost Classifier,0.8040,0.8741,0.7260,0.5979,0.6557,7.660
lightgbm,Light Gradient Boosting Machine,0.7986,0.8737,0.7583,0.5834,0.6593,0.430
gbc,Gradient Boosting Classifier,0.7897,0.8798,0.8087,0.5636,0.6642,2.202
ada,Ada Boost Classifier,0.7840,0.8770,0.8218,0.5540,0.6619,0.788
rf,Random Forest Classifier,0.7830,0.8377,0.6731,0.5656,0.6146,2.542
et,Extra Trees Classifier,0.7817,0.8049,0.6414,0.5668,0.6017,2.814
lr,Logistic Regression,0.7812,0.8784,0.8346,0.5491,0.6623,0.456
dt,Decision Tree Classifier,0.7762,0.7761,0.6306,0.5575,0.5917,0.180
nb,Naive Bayes,0.7745,0.8517,0.7918,0.5423,0.6437,0.124
ridge,Ridge Classifier,0.7743,0.0000,0.8346,0.5397,0.6555,0.160


## CatBoost

### Hyperparameter Tuning

Pycaret의 tune_model 메서드를 사용하여 모델을 hyperparameter를 튜닝합니다.   
이 때, 최적화는 Optuna로 수행합니다.

(여담) CatBoost는 내부 알고리즘이 강력하여 최적화를 통해 퍼포먼스 향상이 많이 이루어지지는 않습니다. 하지만 다른 모형들의 경우는 퍼포먼스 향상이 꽤 이루어질 수 있습니다.

In [ ]:
import optuna

In [ ]:
cb_tuned = tune_model(
    create_model('catboost'),
    n_iter = 50,
    optimize = 'Accuracy',
    search_library='optuna',
    search_algorithm='tpe',
    early_stopping=True,
    early_stopping_max_iters = 10,
    choose_better=True
    )

,Accuracy,AUC,Recall,Prec.,F1
Fold,,,,,
0,0.8198,0.8958,0.7656,0.6218,0.6862
1,0.7989,0.8707,0.7257,0.5886,0.6500
2,0.8001,0.8695,0.7029,0.5938,0.6437
3,0.8066,0.8733,0.7236,0.6032,0.6580
4,0.8049,0.8742,0.7061,0.6030,0.6505
Mean,0.8060,0.8767,0.7248,0.6021,0.6577
Std,0.0075,0.0097,0.0223,0.0113,0.0150


### optimize threshold

많은 분류모형들은 최종 결과로 label이 아닌 확률값을 반환합니다.   
이진분류를 기준으로 라벨을 생성하기 위해서는 기본적으로 확률값이 0.5 이상이라면 1, 아니라면 0과 같은 방식으로 작동합니다.

하지만 이 확률값이 바뀜으로써 더 나은 결과를 얻을 수 있으므로 probability threshold를 최적화하는 메서드가 지원됩니다.

In [ ]:
cb_optimized = optimize_threshold(cb_tuned)

predict_model 메서드는 validation set (1-train_size)에 대해서 예측을 수행합니다.

In [ ]:
predict_model(cb_optimized)

,Model,Accuracy,AUC,Recall,Prec.,F1
0,CatBoost Classifier,0.8202,0.8787,0.5212,0.6845,0.5918


,age,age_Power2,workclass_0,education_0,education_1,education_2,education_3,marital.status_1,occupation_0.0,occupation_1.0,...,native.country_1.0,marital.status_1_multiply_age,age_Power2_multiply_age,age_Power2_multiply_marital.status_1,native.country_1.0_multiply_marital.status_1,marital.status_1_multiply_education_3,hours.per.week_1_multiply_age,target,Label,Score
0,1.210526,1.465374,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,1.0,1.210526,1.773874,1.465374,1.0,1.0,1.210526,0.0,0.0,0.3245
1,0.052632,0.002770,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.000000,0.000146,0.000000,0.0,0.0,0.052632,0.0,0.0,0.9258
2,0.473684,0.224377,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.473684,0.106284,0.224377,1.0,0.0,0.473684,1.0,1.0,0.7965
3,-1.052632,1.108033,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,-0.000000,-1.166351,0.000000,0.0,0.0,-0.000000,0.0,0.0,0.9994
4,-0.684211,0.468144,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,-0.000000,-0.320309,0.000000,0.0,0.0,-0.684211,0.0,0.0,0.9942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2258,0.947368,0.897507,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.947368,0.850270,0.897507,1.0,0.0,0.947368,0.0,0.0,0.7377
2259,0.789474,0.623269,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,1.0,0.789474,0.492054,0.623269,1.0,0.0,0.789474,0.0,1.0,0.7858
2260,0.473684,0.224377,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.473684,0.106284,0.224377,1.0,0.0,0.473684,1.0,1.0,0.7837
2261,1.157895,1.340720,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,1.157895,1.552413,1.340720,1.0,1.0,1.157895,1.0,1.0,0.9442


데이터 저장

In [ ]:
cb_pred = predict_model(cb_optimized, test)
submission['target'] = cb_pred['Label']
submission.to_csv('cb_submission.csv', index=False)

## Voting classifier

Voting classifier는 튜닝된 n개의 모델이 결과를 바탕으로 다시 추정하는 방식입니다.

두가지 방식이 있습니다.
- soft
- hard

우선 각 모델의 hyperparameter를 튜닝합니다.

In [ ]:
tuned_n_model = [tune_model(i, 
                            n_iter = 50, 
                            optimize='Accuracy', 
                            search_library='optuna', 
                            choose_better=True) for i in n_models]
optimized_n_model = [optimize_threshold(i) for i in tuned_n_model]

,Accuracy,AUC,Recall,Prec.,F1
Fold,,,,,
0,0.7980,0.8847,0.8341,0.5741,0.6801
1,0.7878,0.8644,0.8022,0.5614,0.6605
2,0.7800,0.8589,0.7891,0.5501,0.6483
3,0.7774,0.8598,0.7827,0.5469,0.6439
4,0.7815,0.8633,0.7732,0.5538,0.6453
Mean,0.7849,0.8662,0.7963,0.5572,0.6556
Std,0.0074,0.0095,0.0212,0.0097,0.0136


[LightGBM] [Warning] feature_fraction is set=0.43518681372977996, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.43518681372977996
[LightGBM] [Warning] bagging_fraction is set=0.6892679995893641, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6892679995893641
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.43518681372977996, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.43518681372977996
[LightGBM] [Warning] bagging_fraction is set=0.6892679995893641, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6892679995893641
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.43518681372977996, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.43518681372977996
[LightGBM] [Warning] b

blend_models 메서드를 통해 튜닝이 완료된 모델들을 입력하고 validation set에 대해서 예측을 수행합니다.

In [ ]:
voting = blend_models(optimized_n_model, method='soft', choose_better=True)
predict_model(voting)

,Accuracy,AUC,Recall,Prec.,F1
Fold,,,,,
0,0.8153,0.8921,0.7815,0.6102,0.6853
1,0.7972,0.8673,0.7352,0.5843,0.6511
2,0.7923,0.8659,0.7268,0.5759,0.6427
3,0.7996,0.8687,0.7332,0.5885,0.6529
4,0.8008,0.8712,0.7125,0.5939,0.6478
Mean,0.8010,0.8730,0.7379,0.5906,0.6560
Std,0.0077,0.0097,0.0232,0.0114,0.0151


,Model,Accuracy,AUC,Recall,Prec.,F1
0,CatBoost Classifier,0.8069,0.8727,0.712,0.5953,0.6484


,age,age_Power2,workclass_0,education_0,education_1,education_2,education_3,marital.status_1,occupation_0.0,occupation_1.0,...,native.country_1.0,marital.status_1_multiply_age,age_Power2_multiply_age,age_Power2_multiply_marital.status_1,native.country_1.0_multiply_marital.status_1,marital.status_1_multiply_education_3,hours.per.week_1_multiply_age,target,Label,Score
0,1.210526,1.465374,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,1.0,1.210526,1.773874,1.465374,1.0,1.0,1.210526,0.0,1.0,0.7560
1,0.052632,0.002770,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.000000,0.000146,0.000000,0.0,0.0,0.052632,0.0,0.0,0.6615
2,0.473684,0.224377,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.473684,0.106284,0.224377,1.0,0.0,0.473684,1.0,1.0,0.6223
3,-1.052632,1.108033,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,-0.000000,-1.166351,0.000000,0.0,0.0,-0.000000,0.0,0.0,0.9996
4,-0.684211,0.468144,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,-0.000000,-0.320309,0.000000,0.0,0.0,-0.684211,0.0,0.0,0.9898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2258,0.947368,0.897507,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.947368,0.850270,0.897507,1.0,0.0,0.947368,0.0,0.0,0.6840
2259,0.789474,0.623269,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,1.0,0.789474,0.492054,0.623269,1.0,0.0,0.789474,0.0,1.0,0.7643
2260,0.473684,0.224377,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.473684,0.106284,0.224377,1.0,0.0,0.473684,1.0,1.0,0.7226
2261,1.157895,1.340720,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,1.157895,1.552413,1.340720,1.0,1.0,1.157895,1.0,1.0,0.9297


데이터 저장

In [ ]:
voting_pred = predict_model(voting, test)
submission['target'] = voting_pred['Label']
submission.to_csv('voting_submission.csv', index=False)